In [1]:
# Setup
from IPython.display import display, Markdown, Latex, Math
from itertools import combinations

# ISBN 10

## Prüfziffer berechnen

In [2]:
def calculateCheckDigit(isbn10):
    sum = 0
    for i in range(9):
        sum += (i + 1) * int(isbn10[i])
    checkDigit = sum % 11
    if checkDigit == 10:
        return 'X'
    else:
        return str(checkDigit)

In [3]:
isbn = "305501517"
display(Markdown(f"isbn10: {isbn}-**{calculateCheckDigit(isbn)}**"))

isbn10: 305501517-**7**

## Zahlendreher erkennen
![Zahlendreher](images/isbn10_zahlendreher.png)

# GGT, GGT Rückwärts, Lemma von Bezout

## GGT - Euklidischer Algorithmus

In [4]:
def ggT(a, b):
    while b != 0:
        reminder = a % b
        factor = a // b
        print(f"{a} = {factor} * {b} + {reminder}")
        a = b
        b = reminder
    return a

In [5]:
a = 2406
b = 654
display(Markdown(f"ggt({a}, {b}) = **{ggT(a, b)}**"))

2406 = 3 * 654 + 444
654 = 1 * 444 + 210
444 = 2 * 210 + 24
210 = 8 * 24 + 18
24 = 1 * 18 + 6
18 = 3 * 6 + 0


ggt(2406, 654) = **6**

## Euklidischer Algorithmus rückwärts

In [6]:
def ggTRueckwaerts(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = ggTRueckwaerts(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

In [7]:
a = 2406
b = 654
gcd, x, y = ggTRueckwaerts(a, b)
display(Markdown(f"{gcd} = **{x}** * {a} + **{y}** * {b}"))

6 = **28** * 2406 + **-103** * 654

## Lemma von Bezout
![Zahlendreher](images/lemma_bezout.png)

In [8]:
a = 2406
b = 654
print(f"{a} * x - {b} * y = 0\n")
print("=== ggt ===")
gcd = ggT(a, b)
print(f"\nggt({a}, {b}) = {gcd}")
print("=== ggt ===\n")

# allgemeine Lösung xn = (x y)^t
x = b // gcd
y = a // gcd
r = a * x - b * y


display(Math(rf"x = \begin{{pmatrix}} x \\ y \end{{pmatrix}} = x_p + x_n t"))

print(f"allgemeine Lösung:")

display(Math(rf'2406 \cdot \frac{{{b}}}{{{gcd}}} - 654 \cdot \frac{{{a}}}{{{gcd}}} = {a} \cdot {x} - {b} \cdot {y} = {r}'))
allgemeineLoesung = rf"\begin{{pmatrix}} {x} \\ {y} \end{{pmatrix}}"
display(Math(rf'x_n = {allgemeineLoesung}'))

# partikuläre Lösung p = (px py)^t
_, px, py, = ggTRueckwaerts(a, b)
partikiulaereLoesung = rf"\begin{{pmatrix}} {px} \\ {py} \end{{pmatrix}}"
print(f"partikuläre Lösung:")
display(Math(rf'x_p = {partikiulaereLoesung}'))

print("alle Lösungen:")
display(Math(rf'x = \begin{{pmatrix}} x \\ y \end{{pmatrix}} = {partikiulaereLoesung} + {allgemeineLoesung}t'))
display(Math(rf"{a} ({px} + {x}t) - {b} ({py} + {y}t) = {gcd}"))

2406 * x - 654 * y = 0

=== ggt ===
2406 = 3 * 654 + 444
654 = 1 * 444 + 210
444 = 2 * 210 + 24
210 = 8 * 24 + 18
24 = 1 * 18 + 6
18 = 3 * 6 + 0

ggt(2406, 654) = 6
=== ggt ===



<IPython.core.display.Math object>

allgemeine Lösung:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

partikuläre Lösung:


<IPython.core.display.Math object>

alle Lösungen:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Chinesischer Restsatz - simultane Kongruenzen

In [64]:
def chinesischer_restklassensatz(congruences):
    print("Chinesischer Restklassensatz:")
    # Extract divisors and remainders from tuples
    num = [n for n, r in congruences]
    rem = [r for n, r in congruences]

    # Compute product of all numbers
    prod = 1
    for n in num:
        prod *= n
    print(f"mod = {" * ".join(map(str, num))} = {prod}\n")

    terms = []
    result = 0
    # Apply CRT formula
    for n, r in zip(num, rem):
        pp = prod // n
        gcd, inv, _ = ggTRueckwaerts(pp, n)
        inv = inv % n  # Ensure the inverse is positive
        term = r * pp * inv
        result += term
        terms.append(term)
        print(f"Calculating term for modulus {n}:")
        print(f"pp = {pp}, inverse = {inv}")
        print(f"Term = {r} * {pp} * {inv} = {term}\n")

    result = result % prod
    print(f"Final result:")
    print(f"z = {" + ".join(map(str, terms))} = {result} mod {prod}")
    return result

In [68]:
# z = 3 (mod 5)
# z = 4 (mod 7)
# z = 5x + 3 = 7y + 4
congruences = [(5, 3), # mod 5, rest 3
               (7, 4) # mod 7, rest 4
               ]
chinesischer_restklassensatz(congruences)

Chinesischer Restklassensatz:
mod = 5 * 7 = 35

Calculating term for modulus 5:
pp = 7, inverse = 3
Term = 3 * 7 * 3 = 63

Calculating term for modulus 7:
pp = 5, inverse = 3
Term = 4 * 5 * 3 = 60

Final result:
z = 63 + 60 = 18 mod 35


18

In [69]:
# z = 3 (mod 11) 
# z = 6 (mod 8)
# z = 1 (mod 15)
congruences = [
    (11, 3), # mod 11, rest 3
    (8, 6), # mod 8, rest 6
    (15, 1) # mod 15, rest 1
    ]
chinesischer_restklassensatz(congruences)

Chinesischer Restklassensatz:
mod = 11 * 8 * 15 = 1320

Calculating term for modulus 11:
pp = 120, inverse = 10
Term = 3 * 120 * 10 = 3600

Calculating term for modulus 8:
pp = 165, inverse = 5
Term = 6 * 165 * 5 = 4950

Calculating term for modulus 15:
pp = 88, inverse = 7
Term = 1 * 88 * 7 = 616

Final result:
z = 3600 + 4950 + 616 = 1246 mod 1320


1246

# Primfaktorzerlegung

In [10]:
def p_factorization(n):
    i = 2
    lst = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            lst.append(i)
    if n > 1:
        lst.append(n)
    return lst

In [11]:
p_factorization(20)

[2, 2, 5]

## Euler'sche Phi-Funktion
![Euler'sche Phi-Funktion](images/euler_phi.png)

In [12]:
def eulerPhi(n):
    factors = p_factorization(n)
    phi = n
    for factor in factors:
        phi *= 1 - 1/factor
    return int(phi)

In [13]:
eulerPhi(20)

4

# Tunierplan - Rundenturnier - Blockplan - BIBD
![bibd](images/bibd.png)
  

Parameter:
- v: Anzahl der Varietäten
- k: Länge eines Blocks
- λ (lambda): sagt, wie oft jede 2-elementige Teilmenge der Varietäten {1, 2, …, v} im Blockplan vorkommt
- b: Anzahl der Blöcke (b = v über 2)
- r: Wie oft jede Varietät in einem Block vorkommt (r = v - 1)

In [14]:
def Tunierplan(v):
    pairings = []

    for round in range(1, v):
        round_pairings = []

        for x in range(1, v):
            for y in range(x + 1, v):

                if (x + y) % (v - 1) == round % (v - 1):
                    round_pairings.append((x, y))

            if (x + x) % (v - 1) == round % (v - 1):
                round_pairings.append((x, v))

        pairings.append(round_pairings)
    return pairings

In [15]:
v = teams = 8
plan = Tunierplan(v)

table_header = f"|{'|'.join([f" Runde {i} " for i in range(1, len(plan)+ 1)])} |\n"
table_sep = f"|{'|'.join(["----------" for i in range(1, len(plan)+ 1)])}|\n"

table_body = ""
for row in range(len(plan[0])):
    for col in range(len(plan)):
        table_body += f"|{plan[col][row]}"
    table_body += "|\n"

display(Markdown(table_header + table_sep + table_body))

| Runde 1 | Runde 2 | Runde 3 | Runde 4 | Runde 5 | Runde 6 | Runde 7  |
|----------|----------|----------|----------|----------|----------|----------|
|(1, 7)|(1, 8)|(1, 2)|(1, 3)|(1, 4)|(1, 5)|(1, 6)|
|(2, 6)|(2, 7)|(3, 7)|(2, 8)|(2, 3)|(2, 4)|(2, 5)|
|(3, 5)|(3, 6)|(4, 6)|(4, 7)|(5, 7)|(3, 8)|(3, 4)|
|(4, 8)|(4, 5)|(5, 8)|(5, 6)|(6, 8)|(6, 7)|(7, 8)|


# Prinzip der Inklusion und Exklusion – Siebmethode

In [16]:
def calculate_others(total, counts):

    keys = [key for key in counts.keys() if len(key) == 1]
    characteristics = len(counts)
    with_characteristic = 0

    display(Math(f"|\\Omega| = 73 \\; |r| = {len(keys)}"))
    latex = "|\\Omega|"

    for r in range(1, characteristics + 1):
        latex += "+" if r % 2 == 0 else "-"
        latex += "("        
        for combo in combinations(range(characteristics), r):
            count_key = ''.join(chr(65 + i) for i in combo)

            if count_key in counts:
                latex += f"|{count_key}| + " if len(count_key) == 1 else f"|{' \\cap '.join(count_key)}| + "
                with_characteristic += (-1) ** (r + 1) * counts[count_key]

        latex = latex[:-2] + ")"
    
    while '))' in latex:
        latex = latex.replace('))', ')')
    latex += f" = {total - with_characteristic}"
    display(Math(latex))
    
    return total - with_characteristic

In [17]:
total = 55 # 55 Athleten
counts = {
    'A': 35, # Fußball
    'B': 27, # Leichtathletik
    'C': 12, # Judo
    # 'D': 1, # Handball
    'AB': 13, # Fußball, Leichtathletik
    'AC': 7, # Fußball, Judo
    'BC': 5, # Leichtathletik, Judo
    'ABC': 2, # Fußball, Leichtathletik, Judo
    # 'ABCD': 1 # Fußball, Leichtathletik, Judo, Handball
}

print(f"Andere: {calculate_others(total, counts)}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Andere: 4


In [18]:
total = 73
counts = {
    'A': 20,    
    'B': 25,    
    'C': 52,    
    'AB': 7,     
    'AC': 12,    
    'BC': 17,    
    'ABC': 1
}

print(f"Andere: {calculate_others(total, counts)}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Andere: 11
